In [ ]:
%cd ..

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
met_df = pd.read_csv('kelp_data/metadata.csv')

In [ ]:
met_df.head()

In [ ]:
met_df['type'].unique()

In [ ]:
# are there the same number of kelp data points as there are satellite?
kelp_subs = met_df[met_df['type'] == 'kelp']
sat_subs = met_df[met_df['type'] == 'satellite']
len(kelp_subs), len(sat_subs)

In [ ]:
import rasterio

In [ ]:
dataset = rasterio.open('kelp_data/train_satellite/AA498489_satellite.tif')

In [ ]:
{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

In [ ]:
dataset.shape

In [ ]:
ind = dataset.indexes

In [ ]:
dataset.read(ind[2]).shape

In [ ]:
kelp= rasterio.open('kelp_data/train_kelp/AA498489_kelp.tif')

In [ ]:
kelp_subset = met_df[met_df['type'] == 'kelp']

In [ ]:
not_kelp = met_df[~(met_df['type'] == 'kelp')]

In [ ]:
not_kelp

In [ ]:
from tqdm import tqdm
maxmins = {
    'band'+str(i): {'min': 100000,'max': -100000} 
    for i in range(1,6)
}
for _, r in tqdm(not_kelp.iterrows()):
    name = r['filename']
    if r['in_train']:
        img = rasterio.open('kelp_data/train_satellite/'+name)
    else:
        img = rasterio.open('kelp_data/test_satellite/'+name)
    for i in range(1,6):
        band_min = img.read(i)[img.read(i) > -32768].min()
        maxmins['band'+str(i)]['min'] = min(maxmins['band'+str(i)]['min'], band_min)
        band_max = img.read(i)[img.read(i) < 65535].max()
        maxmins['band'+str(i)]['max'] = max(maxmins['band'+str(i)]['max'] , band_max)
    

In [ ]:
maxmins

In [ ]:
from pathlib import Path
empty = 0
for i, k in kelp_subset.iterrows():
    name = k['filename']
    if 
    d = rasterio.open('/home/ziggy/devel/kelp-me/kelp_data/train_kelp/'+name)
    

In [ ]:
len(kelp_subset)

In [ ]:
empty

#### Result: 2000/5635 train are entirely empty

In [ ]:
sat_in_train = (met_df['type'] == 'satellite') * (met_df['in_train'])
sat_train = met_df[sat_in_train]
len(sat_train)


In [ ]:
joined_train = kelp_subset.merge( right = sat_train, on='tile_id' , suffixes=['_kelp','_sat'])

In [ ]:
centerWavelengths = [1650, 860, 650, 550, 470]

In [ ]:
from rasterio.plot import show, show_hist

for i,r in joined_train[:10].iterrows():
    sat_file = rasterio.open('kelp_data/train_satellite/'+r['filename_sat'])
    kelp_file = rasterio.open('kelp_data/train_kelp/'+r['filename_kelp'])
    # for i in sat_file.indexes:
    #     show(sat_file.read(i), transform=sat_file.transform)
    max_val = 65536
    print("#########################################################")
    show(kelp_file.read(1))
    show(sat_file.read([3,4,5])/max_val)
    show(sat_file.read(1))
    show(sat_file.read(2))
    show(sat_file.read(6))
    show(sat_file.read(7))
    
    show_hist(
        sat_file.read(), bins=50, lw=0.0, stacked=False, alpha=0.3,
        histtype='stepfilled', title="Histogram")


### Data Cleaning
- remove invalid pixels (negative int)
- remove cloud mask?
- remove land?
- image augumentation using random masks, transforms, flips, scaling
- 